In [ ]:
import numpy as np
from astropy.units import Quantity

from rls.simulation import Simulation
from rls.data_types import Particle, SimQuantity
from rls.formula.physics import lorentz_factor
from rls.formula.conversions import energy_to_speed
from rls.models import WhistlerAtGradientModel


def generate_particles(
    model,
    Np=100_000,
    W_min=Quantity(0.0, "eV"),
    W_max=Quantity(1000.0, "eV"),
):
    units = model.units
    W_factor = model.units.W_factor
    c = model.units.c
    R = model.R
    
    W_min = SimQuantity(W_factor.user_to_code(W_min), W_min)
    V_min = energy_to_speed(W_min, units)
    W_max = SimQuantity(W_factor.user_to_code(W_max), W_max)
    V_max = energy_to_speed(W_max, units)

    rng = np.random.default_rng()
    V_para = rng.uniform(V_min.code, V_max.code, Np)
    V_perp = rng.uniform(V_min.code, V_max.code, Np)
    phase = rng.uniform(-np.pi, np.pi, Np)
    uz0 = V_para
    ux0 = V_perp * np.cos(phase)
    uy0 = V_perp * np.sin(phase)
    x0 = np.zeros_like(uz0)
    y0 = np.zeros_like(uz0)
    z0 = np.zeros_like(uz0) - 3.0 * R.code
    g0 = lorentz_factor(ux0, uy0, uz0, c.code)
    ICs = Particle(units.electron, 0.0, g0, x0, y0, z0, ux0, uy0, uz0)
    return ICs


def run_sim(ib):
    sim.name = f"vary_Bw_{ib}"
    sim.model.Bw = np.abs(Bw_B0[ib] * sim.model.units.B_factor)
    
    ICs = generate_particles(sim.model)
    Tc = ICs.species.cyclotron_period(sim.model.B0).code
    sim.run(
        initial_conditions=ICs,
        run_time=300.0 * Tc,
        step_size=1e-2 * Tc,
        save_intervals=3,
        log_intervals=20,
    )
    sim.save_data()


Bw_B0 = np.logspace(np.log10(1e-3), np.log10(5e-1), 20)
sim = Simulation(
    model=WhistlerAtGradientModel(
        w_wce=0.05,
        sw=1.75,
        Bh=-0.8,
        B0=-1.0,
        theta=0.0,
    )
)

for ib in range(Nb := Bw_B0.size):
    print(f"Running sim {ib}/{Nb}")
    run_sim(ib)

Running sim 0/20
Pushed 0/30000 steps (0.00%, 0.000013 ms/step, estimated remaining run time = 0.01 min)
Pushed 1500/30000 steps (5.00%, 0.018350 ms/step, estimated remaining run time = 8.72 min)
Pushed 3000/30000 steps (10.00%, 0.019036 ms/step, estimated remaining run time = 8.57 min)
Pushed 4500/30000 steps (15.00%, 0.019755 ms/step, estimated remaining run time = 8.40 min)
Pushed 6000/30000 steps (20.00%, 0.019523 ms/step, estimated remaining run time = 7.82 min)
Pushed 7500/30000 steps (25.00%, 0.019269 ms/step, estimated remaining run time = 7.23 min)
Pushed 9000/30000 steps (30.00%, 0.019089 ms/step, estimated remaining run time = 6.69 min)
Pushed 10500/30000 steps (35.00%, 0.019411 ms/step, estimated remaining run time = 6.31 min)
Pushed 12000/30000 steps (40.00%, 0.019670 ms/step, estimated remaining run time = 5.91 min)
Pushed 13500/30000 steps (45.00%, 0.019888 ms/step, estimated remaining run time = 5.47 min)
Pushed 15000/30000 steps (50.00%, 0.020220 ms/step, estimated rem

In [ ]:
from rls.formula.conversions import cartesian_to_FAC
from astropy.units import Quantity

def process(ib):
    sim.name = f"vary_Bw_{ib}"
    sim.model.Bw = np.abs(Bw_B0[ib] * sim.model.units.B_factor)
    model = sim.model
    sim.load_data()
    units = sim.model.units
    c = units.light_speed
    qe = units.electron.charge
    me = units.electron.mass
    w_wce = model.w_wce
    theta = model.theta
    wpe0 = units.electron.wp(model.n0, units.eps0)
    wce0 = units.electron.wc(model.B0)
    V_factor = (units.L_factor / units.T_factor).to("1000 km/s")
    W_factor = units.W_factor

    t = sim.solutions.t[:]
    g, x, y, z, ux, uy, uz = sim.solutions.as_tuple() 
    tg = t[:, np.newaxis] * np.ones_like(x)
    
    args = model.background_field_args
    _, _, _, B0_x, _, B0_z = model.background_field(tg, x, y, z, *args)
    wce = np.abs(qe.code / me.code * np.sqrt(B0_x**2 + B0_z**2))
    w, k = model.dispersion_relation(wce, w_wce, wpe0.code, c.code)
    V_para, V_perp, W, A = cartesian_to_FAC(g, ux, uy, uz, B0_x, B0_z, model)
    Vi_para = V_para[0, :]
    Vi_perp = V_perp[0, :]
    dW = W[-1, :] - W[0, :]
    dA = A[-1, :] - A[0, :]
    
    resonant_particles = dA.user < 0
    H = np.histogram(
        Vi_para[resonant_particles].user,
        bins=V_bins
    )[0]
    H_dW = np.histogram(
        Vi_para[resonant_particles].user,
        weights=dW[resonant_particles].user,
        bins=V_bins,
    )[0]
    H_dA = np.histogram(
        Vi_para[resonant_particles].user,
        weights=dA[resonant_particles].user,
        bins=V_bins,
    )[0]
    dW_mean = H_dW / H
    dA_mean = H_dA / H

    print(f"Processed {ib}")
    return dW_mean, dA_mean


V_bins = Quantity(np.linspace(-25, 0, 200), "1000 km/s")
V_arr = V_bins[:-1]
Bg, Vg = np.meshgrid(Bw_B0, V_arr, indexing="ij")
dW_mean = np.zeros_like(Bg)
dA_mean = np.zeros_like(Bg)
for ib in range(Bw_B0.size):
    _dW_mean, _dA_mean = process(ib)
    dW_mean[ib, :] = _dW_mean
    dA_mean[ib, :] = _dA_mean

In [ ]:
import astropy.constants.si as c
import astropy.units as u
from tvolib import mpl_utils as mu

iv = 120
Wg = (0.5 * c.m_e * Vg**2).to(u.eV).value

fig, ax = mu.plt.subplots(1, 1)

cax = mu.add_colorbar(ax)
im = ax.pcolormesh(Bg, Vg.value, dW_mean, cmap="jet")
fig.colorbar(im, cax=cax)

ax.set_xscale("log")
ax.set_xlim(Bw_B0[0], Bw_B0[-1])
ax.set_ylim(-20, -5)
ax.axhline(V_arr[iv].value, ls="--", c="k")

fig, ax = mu.plt.subplots(1, 1)

cax = mu.add_colorbar(ax)
im = ax.pcolormesh(Bg, Vg.value, np.abs(dA_mean)**2, cmap="jet")
fig.colorbar(im, cax=cax)

ax.set_xscale("log")
ax.set_xlim(Bw_B0[0], Bw_B0[-1])
ax.set_ylim(-20, -5)
ax.axhline(V_arr[iv].value, ls="--", c="k")

fig, ax = mu.plt.subplots(1, 1)

ax.scatter(Bw_B0, np.abs(dW_mean / Wg)[:, iv]**2, ec="k", fc="w", s=20)
ax.plot(Bw_B0, 0.1 * Bw_B0**2, "--k")
ax.plot(Bw_B0, 0.05 * Bw_B0, "--r")

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(Bw_B0[0], Bw_B0[-1])

mu.plt.show()

fig, ax = mu.plt.subplots(1, 1)

ax.scatter(Bw_B0, np.abs(dA_mean)[:, iv]**2, ec="k", fc="w", s=20)
ax.plot(Bw_B0, 1e5 * Bw_B0**2, "--k")
ax.plot(Bw_B0, 1e4 * Bw_B0, "--r")

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(Bw_B0[0], Bw_B0[-1])

mu.plt.show()